In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize
import distance
from tensorflow.contrib import learn
from sklearn.metrics import mean_squared_error

from lstm import sin_cos, generate_data, lstm_model

DATA = pd.read_table(open("fulldata.csv",encoding='utf-8'),header=None,low_memory=False,sep=';')

In [2]:
X=DATA[[1,5,6,7]]

In [3]:
X

,1,5,6,7
0,subscriber,unix,latitude,longitude
1,580C1941,1475475205,47.603111,19.060024
2,580C1941,1475468165,47.748206,18.504456
3,580C1941,1475496160,47.537370,19.138583
4,580C1941,1475516890,47.537370,19.138583
5,580C1941,1475516230,47.538025,19.149754
6,580C1941,1475496062,47.538025,19.149754
7,580C1941,1475468814,47.748206,18.504456
8,580C1941,1475466683,47.748206,18.504456
9,580C1941,1475466246,47.748206,18.504456


In [4]:
e=X[1].value_counts()

In [5]:
good_list=["58305A05","58209072","582AFE2A","58041E1A","585FBFAB","582D78D2","581F9DFC","5848D0C2","582C911D","583C1DAE","5843CDCC","582169AE","583B0D1A","58437314","585D8BD0","5823DAB9","585D605C","583BC56B","5838039E","585EEE93","5802D3E0","5818E91D","580828AA","58334D62","580F2A7C","5848FFF1","582008D9","5819708F","58120C5C","58199037"]
#reprezentatív mintát véve

In [6]:
if True:
    user="58305A05"
    sel=X.loc[X[1] == user]
    train_x=np.array(sel.as_matrix(columns=[5])[:,0])
    train_y=np.array(sel.as_matrix(columns=[6,7]))
    train_x=train_x.astype(float)
    train_y=train_y.astype(float)
    valid_x=train_x[int(len(train_x)*9/10):len(train_x)]
    train_x=train_x[0:int(len(train_x)*9/10)]
    valid_y=train_y[int(len(train_y)*9/10):len(train_y)]
    train_y=train_y[0:int(len(train_y)*9/10)]

In [7]:
LOG_DIR = './ops_logs/sin_cos'
TIMESTEPS = 300
RNN_LAYERS = [{'num_units': 5}, {'num_units': 2}]
DENSE_LAYERS = None
TRAINING_STEPS = 10000
PRINT_STEPS = TRAINING_STEPS / 10
BATCH_SIZE = 100

In [8]:
regressor = learn.Estimator(model_fn=lstm_model(TIMESTEPS, RNN_LAYERS, DENSE_LAYERS),
                            model_dir=LOG_DIR)

In [9]:
X, y = generate_data(sin_cos, np.linspace(0, 100, 10000, dtype=np.float32), TIMESTEPS, seperate=False)
# create a lstm instance and validation monitor
validation_monitor = learn.monitors.ValidationMonitor(X['val'], y['val'],
                                                      every_n_steps=PRINT_STEPS,
                                                      early_stopping_rounds=1000)


In [10]:
regressor.fit(X['train'], y['train'], 
              monitors=[validation_monitor], 
              batch_size=BATCH_SIZE,
              steps=TRAINING_STEPS)

Instructions for updating:
Use mean_squared_error.


KeyboardInterrupt: 